In [1]:
# an example
from keras.utils.np_utils import to_categorical
from keras.layers import Dense
import pandas as pd
from lineage import Lineage
import numpy as np
from individual import Individual
import pickle
import zipfile


# load the data
#train = pd.read_csv('mnist_train.csv')

# for fashion mnist
with zipfile.ZipFile("fashion-mnist_train.csv.zip","r") as zip_ref:
    zip_ref.extractall()

train = pd.read_csv('fashion-mnist_train.csv')

Y_train = train[['label']]
X_train = train.drop(train.columns[[0]], axis=1)

#Reshape the training set
X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)

#Padding the images by 2 pixels since in the paper input images were 32x32
X_train = np.pad(X_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')

#Standardization
mean_px = X_train.mean().astype(np.float32)
std_px = X_train.std().astype(np.float32)
X_train = (X_train - mean_px)/(std_px)

#One-hot encoding the labels
Y_train = to_categorical(Y_train)

# Here's where we start doing things for the evolutionary algorithm
# First we split off a validation set for assessing fitness
# we'll use the validation set to calculate network fitness

X_val = X_train[0:10000]
Y_val = Y_train[0:10000]

X_train = X_train[10000:]
Y_train = Y_train[10000:]


# We need a few things to establish a population

# what will the output layer look like?
output_config = Dense(units = 10, activation = 'softmax').get_config()

# shape of the input data
input_shape = (32, 32, 1)

# name of the loss function to use
loss = 'categorical_crossentropy'


Using TensorFlow backend.


In [2]:
X_val.shape

(10000, 32, 32, 1)

In [3]:
X_train.shape

(50000, 32, 32, 1)

In [4]:
test = Lineage(input_shape, output_config, loss, X_train, Y_train, X_val, Y_val, trainsize = 50000, valsize = 10000, name="fashion2")

In [ ]:
test.initialise(200)

In [ ]:
test.evolve([50]*10, selection='weighted')
test.evolve([50]*10, selection='rank')
test.evolve([50]*10, selection='rank2')
test.evolve([20]*20, keep=1)
test.evolve([10]*50, keep=1)
test.explore(200, type='climb')